In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re

class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        assert embed_dim % num_heads == 0

        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads

        self.qkv_proj = nn.Linear(embed_dim, embed_dim * 3)
        self.out_proj = nn.Linear(embed_dim, embed_dim)

    def forward(self, x):
        B, T, C = x.size()  # Batch, Tokens, Channels

        qkv = self.qkv_proj(x)
        qkv = qkv.reshape(B, T, 3, self.num_heads, self.head_dim)
        qkv = qkv.permute(2, 0, 3, 1, 4)  # (3, B, heads, T, head_dim)

        q, k, v = qkv[0], qkv[1], qkv[2]

        attn_scores = torch.matmul(q, k.transpose(-2, -1)) / (self.head_dim ** 0.5)
        attn_weights = F.softmax(attn_scores, dim=-1)

        out = torch.matmul(attn_weights, v)
        out = out.transpose(1, 2).contiguous().reshape(B, T, C)
        return self.out_proj(out)

# ---------------------
# Transformer Encoder Layer
# ---------------------
class TransformerEncoderLayer(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.attn = MultiHeadSelfAttention(embed_dim, num_heads)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 4),
            nn.ReLU(),
            nn.Linear(embed_dim * 4, embed_dim)
        )
        self.norm2 = nn.LayerNorm(embed_dim)

    def forward(self, x):
        x = x + self.attn(self.norm1(x))
        x = x + self.ff(self.norm2(x))
        return x

# ---------------------
# Summarizer Wrapper
# ---------------------
class TransformerSummarizer(nn.Module):
    def __init__(self, embed_dim=64, num_heads=4):
        super().__init__()
        self.encoder = TransformerEncoderLayer(embed_dim, num_heads)

    def forward(self, x):
        return self.encoder(x)

# ---------------------
# Preprocessing: Sentence Embeddings using TF-IDF
# ---------------------
def preprocess(text):
    text = re.sub(r'\s+', ' ', text.strip())
    sentences = re.split(r'(?<=[.!?]) +', text)
    return sentences

def vectorize(sentences, dim=64):
    vectorizer = TfidfVectorizer(max_features=dim)
    vecs = vectorizer.fit_transform(sentences).toarray()
    return torch.tensor(vecs, dtype=torch.float32).unsqueeze(0)  # (1, T, D)

# ---------------------
# Main CLI Execution
# ---------------------
if __name__ == "__main__":
    print("🔹 Enter your text below. Press Enter TWICE to submit:\n")
    lines = []
    while True:
        line = input()
        if line.strip() == "":
            break
        lines.append(line)

    input_text = " ".join(lines)
    sentences = preprocess(input_text)

    if len(sentences) < 2:
        print("Not enough content to summarize.")
        exit()

    embed_dim = 64
    x = vectorize(sentences, dim=embed_dim)
    
    model = TransformerSummarizer(embed_dim=embed_dim, num_heads=4)
    with torch.no_grad():
        output = model(x)  # Shape: (1, num_sentences, embed_dim)
        attn_scores = output.norm(dim=-1).squeeze()  # (num_sentences,)

    # Select top 3 most attended sentences
    top_indices = torch.topk(attn_scores, k=min(8, len(sentences))).indices
    selected_sentences = [sentences[i] for i in sorted(top_indices.tolist())]

    print("\n📝 Summary:\n")
    for sent in selected_sentences:
        print("•", sent)


🔹 Enter your text below. Press Enter TWICE to submit:




📝 Summary:

• Leonardo di ser Piero da Vinci[b] (15 April 1452 – 2 May 1519) was an Italian polymath of the High Renaissance who was active as a painter, draughtsman, engineer, scientist, theorist, sculptor, and architect.[3] While his fame initially rested on his achievements as a painter, he has also become known for his notebooks, in which he made drawings and notes on a variety of subjects, including anatomy, astronomy, botany, cartography, painting, and palaeontology.
• Leonardo is widely regarded to have been a genius who epitomised the Renaissance humanist ideal,[4] and his collective works comprise a contribution to later generations of artists matched only by that of his younger contemporary Michelangelo.[3][4] Born out of wedlock to a successful notary and a lower-class woman in, or near, Vinci, he was educated in Florence by the Italian painter and sculptor Andrea del Verrocchio.
• He began his career in the city, but then spent much time in the service of Ludovico Sforza i